# Task 2: Analysing hospital 

**The task is to link the test dataset "Berlin Income" with hospital dataset of 2015 and calculate a bundle of descriptive statistical measures (Minimum, Maximum, Mean, Standard derivation) on bed numbers within a circle of 2 kilometers.**

## Load functions from SoRa R package
This steps are currently required to load all R functions from /R/ directory. In future, the SoRa R package will be installed directly.

In [ ]:
# load R functions from SoRa R Package
path <- "/home/jovyan/R/"
sora_functions  <- dir(path)
for (i in sora_functions) {
  source(paste0(path, i))
}

## Check your changed SORA_API_KEY 
- the environment variable from .Renviron file


In [ ]:
#check environment variable for SORA_API_KEY
Sys.getenv("SORA_API_KEY")

## Load, explore and prepare input survey data

In [ ]:
load("/home/jovyan/data/ber_income.rda")
head(ber_income)

In [ ]:
plot(ber_income$x, ber_income$y,
     xlab = "x", ylab = "y",
     main = "Households in Berlin",
     sub = "crs = 4647"
)
grid()

In [ ]:
# load survey data and load its coordinate reference system (CRS) LAEA (EPSG Code: 3035)
survey_data <- sora_custom(.data = ber_income, crs = 3035)

### Linking
- hint: 
    - to determine the correct linking parameter, you can either use the online web-interface of the datapicker
    - or read the error messages (after starting a linking job), to get an idea which values are valid

In [ ]:
# define geospatial dataset of hospitals (including bed numbers) in 2015
spat_data <- sora_spatial(id = "<--->")

In [ ]:

linking <- sora_linking(
  method = "<--->",
  selection_area = "<--->",
  radius = 2000,
  output = c("mean", "<--->", "<--->", "<--->")
)

In [ ]:
job_id <- sora_request(dataset = survey_data, link_to = spat_data, method = linking)

### Get linking result

In [ ]:
## check, if your current linking job is done
sora_job_done(job_id)

In [ ]:
## check status of all your started linking jobs
sora_jobs()

In [ ]:
## get results
sora_calls <- sora_jobs()
job_id <- sora_calls$job_id[1]

if (sora_job_done(job_id)){
  results <- sora_results(job_id = job_id)
  results <- sora_split_id(results, out_col = c("hid", "syear"))

  ## add column for id
  results$id <- paste(results$hid, results$syear, sep = "_")

  ## print first rows of result
  head(results)
}

In [ ]:
#merge input survey data and result
linked <- merge(ber_income, results, by="id")
head(linked)

### Analyse 

In [ ]:
plot(linked$income, linked$mean, type = "p", main = "income vs average number of hosptial beds", xlab = "Income", ylab = "Number of hospital beds")

We could assume, that the living location of people which higher income has a bigger distance to hospitals then people with lower income.